# Import section

In [4]:
# Data handling
import pandas as pd
import numpy as np

# Model evaluation and metrics
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, precision_score, recall_score

# pyFUME and AUK
from pyfume import SugenoFISTester
from AUK import AUK

# For saving/loading data
import pickle
import os

# Data loading

In [5]:
# Load best hyperparameters from pkl file
best_hyperparameters = pd.read_pickle('model_training/best_hyperparameters.pkl')

# Extract the first row from the 'params' column
params_dict = best_hyperparameters['params'].iloc[0]

# Ensure the 'params' column contains a dictionary
if isinstance(params_dict, dict):
    # Dynamically create variables from the dictionary keys
    for key, value in params_dict.items():
        locals()[key] = value

    # Derive dataframe_name from the params_dict or directly from 'dataframe_name' key
    if 'dataframe_name' in params_dict:
        dataframe_name = params_dict['dataframe_name']

        # Replace 'train' with 'test' in the dataframe name
        dataframe_name = dataframe_name.replace('train', 'test', 1)

        # Construct the .pkl file path
        base_folder = 'feature_selection'
        pkl_file_path = f'{base_folder}/{dataframe_name}.pkl'

        # Load the test dataframe
        test_df = pd.read_pickle(pkl_file_path)

        # Print to verify
        print(f"Loaded dataframe from: {pkl_file_path}")
    else:
        print("Error: 'dataframe_name' not found in params_dict.")
else:
    print("Error: The 'params' column does not contain a dictionary.")

Loaded dataframe from: feature_selection/test_df_all_drop_features_10_PRS.pkl


In [6]:
# drop extra columns that are not needed for the model
df_test_clean = test_df.drop(columns=['eid', 'p130894', 'p130895', 'p53_i0'])

# extract variable names
variable_names = df_test_clean.drop(columns=['target']).columns.tolist()

# Split the test data into features and ground truth
X_test = df_test_clean.drop(columns=['target'])
y_test = df_test_clean['target']

# Convert X_test and y_test to NumPy arrays
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

# Load the saved model
with open('model_training/final_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
    
# Create a tester object to evaluate the model on the test data
tester = SugenoFISTester(model=loaded_model, test_data=X_test, variable_names=variable_names, golden_standard=y_test)

# Predict probabilities for the test data
y_pred_proba = tester.predict()

# Extract the first element of the tuple (the predicted probabilities)
y_pred_proba = y_pred_proba[0]

In [7]:
# Convert probabilities to binary predictions using a threshold (e.g., 0.5)
y_pred = (y_pred_proba > 0.5).astype(int)

In [8]:
# Ensure the index of y_pred_series matches df_test
y_pred_series = pd.Series(y_pred, index=test_df.index, name='y_pred')
y_pred_proba_series = pd.Series(y_pred_proba, index=test_df.index, name='y_pred_proba')

# create copy of df_test

df_test_incl_predictions = test_df.copy()

# Insert the new columns after the 'target' column
df_test_incl_predictions.insert(df_test_incl_predictions.columns.get_loc('target') + 1, 'y_pred', y_pred_series)
df_test_incl_predictions.insert(df_test_incl_predictions.columns.get_loc('target') + 2, 'y_pred_proba', y_pred_proba_series)

In [9]:
df_test_incl_predictions.head()

,p26202,p26204,p26206,p26210,p26212,p26214,p26216,p26218,p26220,p26223,...,p130894,p130895,p53_i0,target,y_pred,y_pred_proba,eid,p130894,p130895,p53_i0
126,1.351496,0.557301,-0.366124,0.634605,0.573798,-1.280260,1.155696,-0.398296,-0.169346,-0.236759,...,2022-05-09,Category_B,2008-10-29,1,1,0.530768,ID_127,2022-05-09,Category_B,2008-10-29
314,0.155567,-1.175251,0.819022,-0.009564,-1.494857,-0.604110,0.990320,1.524634,-1.309131,0.510042,...,2011-01-17,Category_A,2007-11-04,1,1,0.530768,ID_315,2011-01-17,Category_A,2007-11-04
267,-0.447638,1.684454,0.701990,0.502309,1.021844,1.462244,0.619248,0.431618,1.430214,-0.029087,...,2014-08-06,Category_E,2009-01-10,1,1,0.530768,ID_268,2014-08-06,Category_E,2009-01-10
282,1.649647,0.345852,0.844010,0.934727,0.918579,0.649313,-0.840977,0.214394,-0.549006,0.710424,...,2016-11-27,Category_A,2009-01-16,1,1,0.530768,ID_283,2016-11-27,Category_A,2009-01-16
496,0.006435,0.927515,-1.605575,-0.779584,1.279180,0.825457,-0.631817,-0.281542,-1.703244,0.456854,...,2016-11-16,Category_C,2009-06-01,1,1,0.530768,ID_497,2016-11-16,Category_C,2009-06-01


# Calculate Metrics

In [10]:
# Calculate accuracy and AUC
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
print(accuracy)
print(auc)

0.498
0.4838957423318773


In [11]:
# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix_df = pd.DataFrame(conf_matrix, 
                                   index=['Actual 0', 'Actual 1'], 
                                   columns=['Predicted 0', 'Predicted 1'])

In [12]:
confusion_matrix_df.head()

,Predicted 0,Predicted 1
Actual 0,0,251
Actual 1,0,249


In [13]:
# Calculate precision and recall
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Precision: {precision}')
print(f'Recall: {recall}')

Precision: 0.498
Recall: 1.0


In [14]:
# calculate AUK
auk_class = AUK(probabilities=y_pred_proba, labels=y_test)
        
# Calculate AUK
auk = auk_class.calculate_auk()
print(auk)

-0.016119494665006975


In [15]:
# replace columns names 

# Dictionary for mapping old column names to new names
column_mapping = {
    'p2090_i0_Yes': 'Seen doctor (GP) for nerves, anxiety, tension or depression | Yes',
    'p2090_i0_No': 'Seen doctor (GP) for nerves, anxiety, tension or depression | No',
    'p2100_i0_No': 'Seen a psychiatrist for nerves, anxiety, tension or depression | No',
    'p1920_i0_Yes': 'Mood swings | Yes',
    'p1960_i0_No': 'Fed-up feelings | No',
    'p1920_i0_No': 'Mood swings | No',
    'p137_i0': 'Number of treatments/medications taken',
    'p2100_i0_Yes': 'Seen a psychiatrist for nerves, anxiety, tension or depression | Yes',
    'p2050_i0_Not_at_all': 'Frequency of depressed mood in last 2 weeks | Not at all',
    'p1980_i0_Yes': 'Worrier / anxious feelings | Yes'
}

# Rename columns in the DataFrame using the mapping
df_test_incl_predictions.rename(columns=column_mapping, inplace=True)

In [16]:
# check false positives
# Filter df_test_incl_predictions for rows where y_pred is 1 and target is 0
false_positives = df_test_incl_predictions[(df_test_incl_predictions['y_pred'] == 1) & 
                                       (df_test_incl_predictions['target'] == 0)]

false_positives.head(5)

,p26202,p26204,p26206,p26210,p26212,p26214,p26216,p26218,p26220,p26223,...,p130894,p130895,p53_i0,target,y_pred,y_pred_proba,eid,p130894,p130895,p53_i0
359,0.414352,-0.905098,1.501626,-1.597734,0.877157,1.663080,-0.684997,-1.105091,-0.831605,0.155338,...,2011-06-08,Category_E,2008-10-06,0,1,0.530768,ID_360,2011-06-08,Category_E,2008-10-06
435,1.403901,-0.217397,1.078440,1.225851,-1.254674,-1.605330,1.389621,-0.659947,0.152197,-1.047258,...,2017-08-22,Category_E,2008-09-05,0,1,0.530768,ID_436,2017-08-22,Category_E,2008-09-05
321,-0.016384,-0.237620,-0.880448,-0.658492,1.545168,-0.571544,1.819331,0.713031,-0.370547,-0.007350,...,2022-10-16,Category_D,2010-01-09,0,1,0.530768,ID_322,2022-10-16,Category_D,2010-01-09
6,-0.438363,1.014340,-0.342376,1.143688,-0.688061,1.189627,-0.516373,-0.424982,-0.285170,-0.600122,...,2009-04-25,Category_A,2008-05-25,0,1,0.530768,ID_7,2009-04-25,Category_A,2008-05-25
272,-1.590879,0.177807,-1.032322,-0.143212,-0.700782,-0.932963,-0.026424,-0.737831,-0.240465,-0.339874,...,2018-10-20,Category_B,2007-05-15,0,1,0.530768,ID_273,2018-10-20,Category_B,2007-05-15


In [17]:
# Filtered DataFrames for true positives and false positives
true_positives = df_test_incl_predictions[(df_test_incl_predictions['y_pred'] == 1) & (df_test_incl_predictions['target'] == 1)]
false_positives = df_test_incl_predictions[(df_test_incl_predictions['y_pred'] == 1) & (df_test_incl_predictions['target'] == 0)]

# Calculating mean and standard deviation for the first ten columns
mean_std_true_positives = true_positives.iloc[:, :10].agg(['mean', 'std'])
mean_std_false_positives = false_positives.iloc[:, :10].agg(['mean', 'std'])

In [18]:
mean_std_true_positives.head()

,p26202,p26204,p26206,p26210,p26212,p26214,p26216,p26218,p26220,p26223
mean,-0.071980,0.019291,-0.058668,-0.042271,0.059728,0.011819,0.114139,0.078503,-0.040459,-0.039488
std,1.070397,0.998436,1.029158,0.982983,1.022331,0.940814,1.018468,0.981425,0.948349,1.044390


In [19]:
mean_std_false_positives.head()

,p26202,p26204,p26206,p26210,p26212,p26214,p26216,p26218,p26220,p26223
mean,-0.082529,0.037473,-0.042791,-0.083189,-0.072676,-0.056275,0.090147,-0.058848,-0.030073,-0.094678
std,1.010149,1.049509,0.991384,1.029776,0.975519,0.955827,1.023628,1.025980,0.943549,1.067135


# export predictions and ground truth

In [20]:
# Define the folder path
folder_name = "model_evaluation"

# Check if the folder exists, if not, create it
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created.")
else:
    print(f"Folder '{folder_name}' already exists.")

Folder 'model_evaluation' created.


In [21]:
# export predictions
np.savez('model_evaluation/predictions.npz', y_pred=y_pred, y_pred_proba=y_pred_proba, y_test=y_test)